In [ ]:
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import *

## Chilean wild cat app

<img src="./images/gato_andino.jpg" alt="Drawing" style="width: 200px;"/>

Hello there! Pick a photo to classify your chilean wildcat!
1. Gato Andino
1. Gato de las pampas
1. Gato Huiña

In [116]:
# load the model in learn_inf
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner(path/'wildcat_model.pkl')

In [125]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [126]:
# listener method:
btn_run = widgets.Button(description='Classify')

In [127]:
# listener method:                      
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(128,128))
    pred, idx, probs = learn_inf.predict(img)
    lbl_pred.value = f'Predicted {pred} with {probs[idx]*100:.01f}% certainty'
    
btn_run.on_click(on_click_classify)

In [128]:
VBox([
     widgets.Label('Classify your chilean wildcat!'),
     btn_upload,
     btn_run,
     out_pl,
     lbl_pred
     ])